In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization)
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE


In [7]:
TRAIN_DIR = '../../data/train'  # folder with subfolders per class
TEST_DIR = '../../data/test'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [9]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True  # randomize batch order
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False  # for consistent evaluation
)

num_classes = train_generator.num_classes
print(f"Detected {num_classes} classes.")

class_indices = train_generator.class_indices  # eg. {'4011': 0, '4012': 1, ...}
print("Class Indices:", class_indices)

# # We'll build an array of class labels from the filenames
# train_labels = train_generator.classes  # an array of integers, e.g. [0, 0, 0, 1, 1, 2, ...]
# cw = class_weight.compute_class_weight(
#     class_weight='balanced',
#     classes=np.unique(train_labels),
#     y=train_labels
# )
# class_weights = dict(enumerate(cw))
# print("Class weights:", class_weights)

X_list, y_list = [], []

# We iterate over the entire train_generator
# The 'steps_per_epoch' is the total number of batches
steps_per_epoch = len(train_generator)

print("Loading all training images into memory...")
for i in range(steps_per_epoch):
    X_batch, y_batch = train_generator[i]
    X_list.append(X_batch)
    y_list.append(y_batch)

X_full = np.concatenate(X_list, axis=0)  # shape: (N, 224, 224, 3)
y_full = np.concatenate(y_list, axis=0)  # shape: (N, num_classes)

print("X_full shape:", X_full.shape)
print("y_full shape:", y_full.shape)

# ------------------------------------------------
# 3. Convert one-hot labels to integer labels
# ------------------------------------------------
y_int = np.argmax(y_full, axis=1)  # shape: (N,)

# Optional: Look at original class distribution
unique, counts = np.unique(y_int, return_counts=True)
print("Original class distribution:", dict(zip(unique, counts)))

# ------------------------------------------------
# 4. Apply SMOTE to oversample the minority class
#    (Flatten images -> SMOTE -> reshape)
# ------------------------------------------------
print("Applying SMOTE...")
X_flat = X_full.reshape((X_full.shape[0], -1))  # shape: (N, 224*224*3)

smote = SMOTE(random_state=42)
X_sm, y_sm = smote.fit_resample(X_flat, y_int)

# Reshape the SMOTEd data back to image format
X_sm = X_sm.reshape((X_sm.shape[0], IMG_SIZE[0], IMG_SIZE[1], 3))

# Convert the integer labels back to one-hot
y_sm_onehot = tf.keras.utils.to_categorical(y_sm, num_classes=num_classes)

print("After SMOTE, X_sm shape:", X_sm.shape)
print("After SMOTE, y_sm_onehot shape:", y_sm_onehot.shape)
unique_sm, counts_sm = np.unique(y_sm, return_counts=True)
print("New class distribution:", dict(zip(unique_sm, counts_sm)))


Found 5181 images belonging to 26 classes.
Found 1307 images belonging to 26 classes.
Detected 26 classes.
Class Indices: {'4011': 0, '4015': 1, '4088': 2, '4196': 3, '7020097009819': 4, '7020097026113': 5, '7023026089401': 6, '7035620058776': 7, '7037203626563': 8, '7037206100022': 9, '7038010009457': 10, '7038010013966': 11, '7038010021145': 12, '7038010054488': 13, '7038010068980': 14, '7039610000318': 15, '7040513000022': 16, '7040513001753': 17, '7040913336684': 18, '7044610874661': 19, '7048840205868': 20, '7071688004713': 21, '7622210410337': 22, '90433917': 23, '90433924': 24, '94011': 25}
Loading all training images into memory...
X_full shape: (5181, 224, 224, 3)
y_full shape: (5181, 26)
Original class distribution: {np.int64(0): np.int64(187), np.int64(1): np.int64(387), np.int64(2): np.int64(280), np.int64(3): np.int64(379), np.int64(4): np.int64(289), np.int64(5): np.int64(97), np.int64(6): np.int64(158), np.int64(7): np.int64(41), np.int64(8): np.int64(75), np.int64(9): n

MemoryError: Unable to allocate 6.71 GiB for an array with shape (11960, 150528) and data type float32

In [ ]:
smote_datagen = ImageDataGenerator(rescale=1./255)
train_smote_generator = smote_datagen.flow(
    X_sm,
    y_sm_onehot,
    batch_size=BATCH_SIZE,
    shuffle=True
)

In [ ]:
model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(32, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate = 1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 222, 222, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 109, 109, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 52, 52, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 43264)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │    11,075,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 26)             │         3,354 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,136,122 (42.48 MB)

 Trainable params: 11,135,898 (42.48 MB)

 Non-trainable params: 224 (896.00 B)

In [ ]:
# train the model

history = model.fit(
    train_smote_generator,   # <--- use the new generator
    epochs=EPOCHS,
    validation_data=test_generator,
    # class_weight=class_weights  # Typically no need after SMOTE
)

plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.show()

test_loss, test_acc = model.evaluate(test_generator, verbose=0)
print(f"Test Accuracy: {test_acc:.3f}")
print(f"Test Loss: {test_loss:.3f}")

model.save('my_classification_model.h5')
print("Model saved to my_classification_model.h5")

NameError: name 'model' is not defined

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')

plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose=0)
print(f"Test Accuracy: {test_acc:.3f}")
print(f"Test Loss: {test_loss:.3f}")

model.save('my_classification_model.h5')
print("Model saved to my_classification_model.h5")